In [37]:
import pandas as pd

In [38]:
UNNORMALIZED_RPKM_PATH = "/data/yosef3/scratch/ruchir/data/geuvadis/analysis_results/GD660.GeneQuantRPKM.txt.gz"
NORMALIZED_RPKM_PATH = "/data/yosef3/scratch/ruchir/data/geuvadis/analysis_results/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.gz"
METADATA_PATH = "/data/yosef3/users/ruchir/pgp_uq/data/E-GEUV-1.sdrf.txt"

In [39]:
unnormalized_rpkm_df = pd.read_csv(UNNORMALIZED_RPKM_PATH, sep="\t", index_col=0)
normalized_rpkm_df = pd.read_csv(NORMALIZED_RPKM_PATH, sep="\t", index_col=0)
metadata_df = pd.read_csv(METADATA_PATH, sep="\t", index_col=0)

In [40]:
unnormalized_rpkm_df.head()

,Gene_Symbol,Chr,Coord,HG00096.1.M_111124_6,HG00097.7.M_120219_2,HG00099.1.M_120209_6,HG00099.5.M_120131_3,HG00100.2.M_111215_8,HG00101.1.M_111124_4,HG00102.3.M_120202_8,HG00103.4.M_120208_3,HG00104.1.M_111124_5,HG00105.1.M_120209_7,HG00105.3.M_120223_6,HG00106.4.M_120208_5,HG00108.7.M_120219_2,HG00109.1.M_120209_4,HG00109.3.M_120202_5,HG00110.2.M_120131_2,HG00111.1.M_120209_8,HG00111.2.M_111215_4,HG00112.6.M_120119_2,HG00114.1.M_120209_3,HG00114.6.M_120217_1,HG00115.6.M_120119_1,HG00116.2.M_120131_1,HG00117.1.M_111124_2,HG00117.1.M_120209_1,HG00117.2.M_111216_4,HG00117.3.M_120202_6,HG00117.4.M_120208_4,HG00117.5.M_120131_3,HG00117.6.M_120217_1,HG00117.7.M_120219_4,HG00118.4.M_120208_5,HG00119.1.M_120209_3,HG00119.2.M_111216_6,HG00120.3.M_120202_2,HG00121.1.M_111124_7,HG00122.6.M_120119_1,HG00123.4.M_120208_7,HG00124.3.M_120223_7,HG00125.1.M_111124_6,HG00126.1.M_111124_8,HG00127.1.M_111124_2,HG00128.1.M_111124_6,HG00129.4.M_120208_8,HG00130.5.M_120131_7,HG00131.1.M_120209_8,HG00131.2.M_111215_5,HG00132.2.M_111215_4,HG00133.1.M_120209_5,HG00133.2.M_111216_2,HG00134.1.M_120209_3,HG00134.6.M_120119_6,HG00135.3.M_120202_8,HG00136.4.M_120208_7,HG00137.1.M_120209_6,HG00137.6.M_120217_1,HG00138.1.M_120209_2,HG00138.5.M_120131_5,HG00139.7.M_120219_1,HG00141.5.M_120131_3,HG00142.1.M_120209_7,HG00142.4.M_120208_6,HG00143.1.M_120209_7,HG00143.7.M_120219_2,HG00145.6.M_120119_1,HG00146.2.M_111216_1,HG00148.3.M_120202_6,HG00149.1.M_111124_6,HG00150.4.M_120208_7,HG00151.3.M_120202_4,HG00152.7.M_120219_3,HG00154.5.M_120131_7,HG00155.1.M_111124_2,HG00156.4.M_120208_1,HG00157.5.M_120131_3,HG00158.1.M_111124_8,HG00159.7.M_120219_6,HG00160.3.M_120202_1,HG00171.3.M_120202_2,HG00173.3.M_120202_1,HG00174.4.M_120208_3,HG00176.4.M_120208_2,HG00177.4.M_120208_8,HG00178.4.M_120208_8,HG00179.1.M_111124_8,HG00180.1.M_111124_8,HG00181.4.M_120208_4,HG00182.1.M_111124_4,HG00183.1.M_120209_4,HG00183.5.M_120131_5,HG00185.1.M_111124_5,HG00186.3.M_120202_5,HG00187.1.M_120209_2,HG00188.1.M_111124_3,HG00189.1.M_111124_5,HG00231.2.M_111216_2,HG00232.7.M_120219_3,HG00233.2.M_111215_8,HG00234.7.M_120219_4,HG00235.1.M_111124_1,HG00236.5.M_120131_5,HG00238.5.M_120131_5,HG00239.7.M_120219_4,HG00240.2.M_111215_5,HG00242.3.M_120202_7,HG00243.4.M_120208_2,HG00244.5.M_120131_7,HG00245.4.M_120208_6,HG00246.7.M_120219_4,HG00247.3.M_120202_1,HG00249.3.M_120202_4,HG00250.1.M_111124_4,HG00251.2.M_111216_8,HG00252.1.M_111124_5,HG00253.5.M_120131_7,HG00255.2.M_111216_1,HG00256.2.M_111215_7,HG00257.4.M_120208_5,HG00258.1.M_111124_6,HG00259.5.M_120131_3,HG00260.5.M_120131_7,HG00261.6.M_120119_2,HG00262.2.M_111215_8,HG00263.6.M_120119_3,HG00264.6.M_120119_5,HG00265.2.M_111215_4,HG00266.6.M_120119_3,HG00267.4.M_120208_6,HG00268.5.M_120131_3,HG00269.2.M_111216_7,HG00271.2.M_111215_7,HG00272.7.M_120219_7,HG00273.3.M_120202_7,HG00274.6.M_120119_1,HG00275.4.M_120208_8,HG00276.2.M_111215_8,HG00277.1.M_120209_5,HG00277.3.M_120202_6,HG00278.1.M_111124_5,HG00280.1.M_111124_6,HG00281.1.M_111124_3,HG00282.2.M_111216_6,HG00284.1.M_111124_6,HG00285.3.M_120202_5,HG00306.1.M_111124_1,HG00308.3.M_120202_3,HG00309.7.M_120219_7,HG00310.4.M_120208_3,HG00311.4.M_120208_6,HG00312.7.M_120219_6,HG00313.1.M_120209_2,HG00313.2.M_111216_6,HG00315.1.M_120209_5,HG00315.2.M_111215_5,HG00319.4.M_120208_7,HG00320.1.M_111124_1,HG00321.1.M_120209_6,HG00321.2.M_111216_2,HG00323.1.M_111124_4,HG00324.1.M_111124_2,HG00325.1.M_120209_3,HG00325.5.M_120131_1,HG00326.1.M_111124_7,HG00327.3.M_120202_4,HG00328.1.M_120209_2,HG00328.2.M_111215_4,HG00329.5.M_120131_3,HG00330.1.M_120209_2,HG00331.7.M_120219_8,HG00332.6.M_120217_1,HG00334.2.M_111216_5,HG00335.2.M_111216_5,HG00336.1.M_120209_2,HG00337.5.M_120131_5,HG00338.1.M_120209_3,HG00338.2.M_111215_8,HG00339.4.M_120208_1,HG00341.2.M_111216_7,HG00342.1.M_111124_1,HG00343.1.M_111124_5,HG00344.4.M_120208_2,HG00345.1.M_120209_1,HG00346.2.M_120131_1,HG00349.1.M_120209_4,HG00349.4.M_120208_4,HG00350.3.M_120202_3,HG00351.6.M_120119_5,HG00353.1.M_111124_6,HG00355.1.M_111124_8,HG00355.

In [41]:
assay_names = set(metadata_df["Assay Name"].unique())
unnormalized_rpkm_df = pd.concat(
    (
        unnormalized_rpkm_df.iloc[:, :3],
        unnormalized_rpkm_df[
            [c for c in unnormalized_rpkm_df.columns if c in assay_names]
        ],
    ),
    axis=1,
)
print(unnormalized_rpkm_df.shape)

(53934, 465)


In [42]:
# Rename columns
unnormalized_rpkm_df.columns = [c.split(".")[0] for c in unnormalized_rpkm_df.columns]

In [43]:
# Remove rows where at least 50% of samples are equal to 0
sample_cols = [
    c for c in unnormalized_rpkm_df.columns if c.startswith("HG") or c.startswith("NA")
]
n_zero_samples = (unnormalized_rpkm_df[sample_cols] == 0).sum(axis=1)
unnormalized_rpkm_df = unnormalized_rpkm_df.loc[n_zero_samples < 0.5 * len(sample_cols)]
assert unnormalized_rpkm_df.shape == normalized_rpkm_df.shape
assert set(unnormalized_rpkm_df.index) == set(normalized_rpkm_df.index)

In [44]:
unnormalized_rpkm_df.to_csv("unnormalized_rpkm_processed.csv.gz")